### <p style="font-family: Arial; color: gold; font-weight: bold;">**create by Tom Tan in 8.30.2024** </p>
##### Now one notebook will deal with all the prefix, the downside is that the define properties will be the same for all the prefix.

***
# **1. Imports**

In [1]:
import os, re, glob
import pandas as pd

import get_properties_functions_for_WI as gp

common_structure_folder = "1.common_structure"
log_files_folder = "2.log_files"
sdf_files_folder = "3.sdf_files"
temp_folder = "temp"
atom_mappings_folder = "4.atom_mappings"
output_folder = "5.postprocessed_results"

***
# **2. Import the atom map from preprocess notebook**
### <p style="font-family: Arial; color: gold; font-weight: bold;"> **Grep all available prefix based on the log file** </p>

In [2]:
prefixs = {}
for file in glob.glob("*.xlsx", root_dir=atom_mappings_folder):
    key = re.search(r"^(\D+)_atom_map", file)
    if key and key.group(1) in prefixs:
        prefixs[key.group(1)].append(file)
    else:
        prefixs[key.group(1)] = [file]

In [3]:
atom_map_df_all = {}
for prefix in prefixs:
    atom_mappings = atom_mappings_folder + os.sep + prefix + "_atom_map.xlsx"
    atom_map_df = pd.read_excel(
        atom_mappings, "Sheet1", index_col=0, header=0, engine="openpyxl"
    )
    # add the log_files_folder before all the log_names cells
    atom_map_df["log_name"] = log_files_folder + os.sep + atom_map_df["log_name"]
    atom_map_df_all[prefix] = atom_map_df.copy(deep=True)
    print(f"prefix: {prefix}")
    display(atom_map_df.head(3))

prefix: pyrdz


,log_name,N3,N4,C5,C6,C7,C2,C1,H1
0,2.log_files\pyrdz1_conf-1,N7,N6,C5,C4,C3,C2,C1,H8
1,2.log_files\pyrdz2_conf-1,C3,N4,N5,C6,C7,C2,C1,H8
2,2.log_files\pyrdz3_conf-1,N8,N7,C6,C5,C4,C3,C2,H12


prefix: pyrd


,log_name,C3,C4,N5,C6,C7,C2,C1,H1
0,2.log_files\pyrd1_conf-1,C5,C4,N3,C11,C10,C2,C1,H12
1,2.log_files\pyrd2_conf-1,C11,N10,C9,C4,C3,C2,C1,H12
2,2.log_files\pyrd3_conf-1,C3,C4,N5,C6,C7,C2,C1,H12


prefix: pyrmd


,log_name,N3,C4,C5,C6,N7,C2,C1,H1
0,2.log_files\pyrmd1_conf-1,N11,C10,C5,C4,N3,C2,C1,H12
1,2.log_files\pyrmd2_conf-1,C10,N9,C8,C3,N11,C2,C1,H12
2,2.log_files\pyrmd3_conf-1,N12,C11,C10,C5,N4,C3,C2,H16


prefix: pyrz


,log_name,C3,N4,C5,C6,N7,C2,C1,H1
0,2.log_files\pyrz1_conf-1,C11,N10,C5,C4,N3,C2,C1,H12
1,2.log_files\pyrz2_conf-1,C12,N11,C10,C5,N4,C3,C2,H16
2,2.log_files\pyrz2_conf-2,C12,N11,C10,C5,N4,C3,C2,H16


# **3. Define Properties to Collect**
### <p style="font-family: Arial; color: gold"> !!!User input required, Change/comment the properties block to the one you want to collect. </p>

In [4]:
for prefix, _ in atom_map_df_all.items():
    print(f"processing prefix: {prefix}")
    pd.set_option("display.max_columns", None)
    # ---------------GoodVibes Engergies---------------
    # uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
    # calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
    # inputs: dataframe, temperature
    df = atom_map_df_all[prefix].copy(deep=True)
    df = gp.get_goodvibes_e(df, 298.15)

    # ---------------Frontier Orbitals-----------------
    # E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
    df = gp.get_frontierorbs(df)

    # ---------------Polarizability--------------------
    # Exact polarizability
    df = gp.get_polarizability(df)

    # ---------------Dipole----------------------------
    # Total dipole moment magnitude in Debye
    df = gp.get_dipole(df)

    # ---------------Volume----------------------------
    # Molar volume
    # requires the Gaussian keyword = "volume" in the .com file
    df = gp.get_volume(df)

    # ---------------SASA------------------------------
    # Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
    df = gp.get_SASA(df)

    # ---------------NBO-------------------------------
    # natural charge from NBO
    # requires the Gaussian keyword = "pop=nbo7" in the .com file
    nbo_list = ["C1", "C2"]
    df = gp.get_nbo(df, nbo_list)

    # ---------------NMR-------------------------------
    # isotropic NMR shift
    # requires the Gaussian keyword = "nmr=giao" in the .com file
    nmr_list = ["C1", "C2"]
    df = gp.get_nmr(df, nmr_list)

    # ---------------Distance--------------------------
    # distance between 2 atoms
    dist_list_of_lists = [["C1", "C2"]]
    df = gp.get_distance(df, dist_list_of_lists)

    # ---------------Angle-----------------------------
    # angle between 3 atoms
    # angle_list_of_lists = [["C5", "N1", "C1"]]
    # df = gp.get_angles(df, angle_list_of_lists)

    # ---------------Dihedral--------------------------
    # dihedral angle between 4 atoms
    # dihedral_list_of_lists = [["C4", "C5", "N1", "C1"], ["C2", "C1", "N1", "C5"]]
    # df = gp.get_dihedral(df, dihedral_list_of_lists)

    # ---------------Vbur Scan-------------------------
    # uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
    # inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
    # if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
    vbur_list = ["C1", "C2"]
    df = gp.get_vbur_scan(df, vbur_list, 2, 2, 0.5)

    # ---------------Sterimol morfeus------------------
    # uses morfeus to calculate Sterimol L, B1, and B5 values
    # NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
    sterimol_list_of_lists = [["C1", "C2"]]
    df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists)

    # ---------------Buried Sterimol-------------------
    # uses morfeus to calculate Sterimol L, B1, and B5 values within a given sphere of radius r_buried
    # atoms outside the sphere + 0.5 vdW radius are deleted and the Sterimol vectors are calculated
    # for more information: https://kjelljorner.github.io/morfeus/sterimol.html
    # inputs: dataframe, list of atom pairs, r_buried
    # sterimol_list_of_lists = [["C1", "C2"]]
    # df = gp.get_buried_sterimol(df, sterimol_list_of_lists, 5.5)

    # ---------------Sterimol DBSTEP-------------------
    # uses DBSTEP to calculate Sterimol L, B1, and B5 values
    # default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
    # more info here: https://github.com/patonlab/DBSTEP
    # NOTE: this takes longer than the morfeus function (recommendation: only use this if you need Sterimol2Vec)
    # sterimol_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
    # df = gp.get_sterimol_dbstep(df, sterimol_list_of_lists)

    # ---------------Sterimol2Vec----------------------
    # uses DBSTEP to calculate Sterimol Bmin and Bmax values at intervals from 0 to end_radius, with a given step_size
    # default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
    # more info here: https://github.com/patonlab/DBSTEP
    # inputs: dataframe, list of atom pairs, end_radius, and step_size
    # sterimol2vec_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
    # df = gp.get_sterimol2vec(df, sterimol2vec_list_of_lists, 1, 1.0)

    # ---------------Pyramidalization------------------
    # uses morfeus to calculate pyramidalization based on the 3 atoms in closest proximity to the defined atom
    # collects values based on two definitions of pyramidalization
    # details on these values can be found here: https://kjelljorner.github.io/morfeus/pyramidalization.html
    pyr_list = ["C1"]
    df = gp.get_pyramidalization(df, pyr_list)

    # ---------------Plane Angle-----------------------
    # !plane angle between 2 planes (each defined by 6 atoms)
    # planeangle_list_of_lists = [["N1", "C1", "C5"], ["C2", "C3", "C4"]]
    # df = gp.get_planeangle(df, planeangle_list_of_lists)

    # --------------LP energy - custom from first cell---------------
    # lp_list = ["N1"]
    # df = gp.get_one_lp_energy(df, lp_list)

    # ---------------Time----------------------------------
    # returns the total CPU time and total Wall time (not per subjob) because we are pioneers
    # if used in summary df, will give the average (not Boltzmann average) in the Boltzmann average column
    # df = gp.get_time(df)

    # ---------------ChelpG----------------------------
    # ChelpG ESP charge
    # requires the Gaussian keyword = "pop=chelpg" in the .com file
    # a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
    # df = gp.get_chelpg(df, a_list)

    # ---------------Hirshfeld-------------------------
    # Hirshfeld charge, CM5 charge, Hirshfeld atom dipole
    # requires the Gaussian keyword = "pop=hirshfeld" in the .com file
    # a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
    # df = gp.get_hirshfeld(df, a_list)

    # !new functions below!
    # ---------------Natural Bond Order (total/covalent/ionic)-------------------------
    # Natural Bond Order (total/covalent/ionic) between 2 atoms, might return non-numerical values
    # requires Natural Resonance Theory Analysis in Gaussian input file ("$nbo nrt $end" in the .com file)
    natural_bond_order_list = [["C1", "C2"]]
    df = gp.get_natural_bond_order(df, natural_bond_order_list)

    # ---------------Natural Atomic Valencies, Electron Counts, and Charges-------------------------
    # Natural Atomic Valencies, Electron Counts, and Charges of a atom
    # requires Natural Resonance Theory Analysis in Gaussian input file ("$nbo nrt $end" in the .com file)
    natural_atomic_valencies_list = ["C1", "C2"]
    df = gp.get_natural_atomic_valencies(df, natural_atomic_valencies_list)

    display(df)
    # copy the changes back to atom_map_df_all
    atom_map_df_all[prefix] = df.copy(deep=True)
    
# delete the "Goodvibes_output.dat" temp file
if os.path.exists("Goodvibes_output.dat"):
    os.remove("Goodvibes_output.dat")

processing prefix: pyrdz
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
NMR function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
Pyramidalization function has completed for ['C1']
Natural Bond Order function has completed for [['C1', 'C2']]
Natural Atomic Valencies function has completed for ['C1', 'C2']


,log_name,N3,N4,C5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NMR_shift_C1,NMR_shift_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrdz1_conf-1,N7,N6,C5,C4,C3,C2,C1,H8,-303.585861,0.103728,-303.475215,0.036948,0.036617,-303.512163,-303.511832,298.15,-0.30541,-0.02096,-0.163185,0.28445,0.04681,68.2186,40.4381,4.1200,891.992,251.280909,337.591570,0.933080,-0.64306,0.17296,161.3275,-2.8831,1.50381,93.055914,95.674070,6.740781,1.843830,3.254861,5.896483,0.798456,0.9572,0.9355,0.0217,3.2503,0.6982,0.0083,3.6184,0.3130,-0.0686
1,2.log_files\pyrdz2_conf-1,C3,N4,N5,C6,C7,C2,C1,H8,-303.584349,0.103775,-303.473550,0.038012,0.037077,-303.511562,-303.510627,298.15,-0.30686,-0.01756,-0.162210,0.28930,0.04548,68.0596,38.9528,4.7174,870.504,250.420802,337.307042,0.935759,-0.63618,-0.01824,166.3383,27.3979,1.50329,93.188275,96.791064,6.241607,1.744658,3.257621,6.046751,0.813583,1.0035,0.9597,0.0438,3.2565,0.7099,0.0241,3.8146,0.1228,-0.0598
2,2.log_files\pyrdz3_conf-1,N8,N7,C6,C5,C4,C3,C2,H12,-342.892888,0.132713,-342.752044,0.040598,0.039684,-342.792642,-342.791728,298.15,-0.30467,-0.02068,-0.162675,0.28399,0.04659,80.4574,42.2635,3.9793,919.812,279.930899,387.383183,0.918038,-0.42954,0.17965,150.4973,-10.2234,1.50694,96.975077,96.410124,6.737827,1.700000,3.271550,5.855227,0.794214,0.9456,0.9300,0.0155,3.4788,0.4751,-0.0005,3.5967,0.3174,-0.0708
3,2.log_files\pyrdz3_conf-2,N8,N7,C6,C5,C4,C3,C2,H12,-342.892412,0.132520,-342.751758,0.040450,0.039608,-342.792208,-342.791366,298.15,-0.30336,-0.01972,-0.161540,0.28364,0.04600,80.3406,45.7824,3.9683,1070.321,279.760945,386.201101,0.916726,-0.43189,0.18124,153.0401,-4.7252,1.51274,97.368931,95.583678,6.757251,1.974166,3.259636,6.219414,0.829867,0.9482,0.9361,0.0121,3.4817,0.4722,-0.0026,3.6043,0.3146,-0.0662
4,2.log_files\pyrdz3_conf-3,N8,N7,C6,C5,C4,C3,C2,H12,-342.892010,0.132648,-342.751259,0.041106,0.039850,-342.792365,-342.791109,298.15,-0.30330,-0.02016,-0.161730,0.28314,0.04619,80.2948,45.6515,4.0170,1145.081,277.990834,384.525806,0.919893,-0.43427,0.17830,154.4339,-8.8372,1.51290,97.352789,95.645015,6.738841,1.974309,3.249029,6.316067,0.839140,0.9513,0.9318,0.0195,3.4757,0.4778,-0.0014,3.6064,0.3090,-0.0703


processing prefix: pyrd
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
NMR function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
Pyramidalization function has completed for ['C1']
Natural Bond Order function has completed for [['C1', 'C2']]
Natural Atomic Valencies function has completed for ['C1', 'C2']


,log_name,C3,C4,N5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NMR_shift_C1,NMR_shift_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrd1_conf-1,C5,C4,N3,C11,C10,C2,C1,H12,-441.215801,0.163282,-441.043098,0.043006,0.042660,-441.086104,-441.085759,298.15,-0.28447,-0.02605,-0.155260,0.25842,0.04664,123.1820,100.5670,1.8012,1407.377,325.682431,470.864551,0.898712,-0.64364,0.23284,157.7232,0.4339,1.50642,93.055914,95.680527,8.877422,1.930717,4.598011,5.912687,0.800030,0.9552,0.9433,0.0120,3.2691,0.6784,0.0006,3.5377,0.4177,-0.0308
1,2.log_files\pyrd2_conf-1,C11,N10,C9,C4,C3,C2,C1,H12,-441.212895,0.163425,-441.040035,0.043108,0.042693,-441.083144,-441.082728,298.15,-0.28330,-0.02685,-0.155075,0.25645,0.04689,123.1750,102.4270,2.2201,1371.628,324.632344,470.260010,0.900847,-0.62780,-0.06631,164.9883,32.8402,1.50530,93.356147,96.742639,9.049449,1.924579,4.320338,6.092525,0.818093,1.0016,0.9637,0.0378,3.2747,0.6890,-0.0078,3.7837,0.1419,-0.0467
2,2.log_files\pyrd3_conf-1,C3,C4,N5,C6,C7,C2,C1,H12,-441.213933,0.163893,-441.040840,0.042181,0.042142,-441.083020,-441.082982,298.15,-0.28400,-0.02937,-0.156685,0.25463,0.04821,121.0120,89.0432,2.5383,1394.325,318.672363,464.399716,0.910055,-0.64103,0.05643,164.6385,16.5349,1.50460,94.040548,96.745868,7.014147,1.887599,5.721767,6.067483,0.815627,1.0039,0.9662,0.0377,3.2666,0.6948,0.0212,3.8102,0.1638,-0.0203
3,2.log_files\pyrd4_conf-1,C5,C4,N3,C11,C6,C2,C1,H12,-441.213784,0.163746,-441.040797,0.042286,0.042230,-441.083083,-441.083027,298.15,-0.27815,-0.02795,-0.153050,0.25020,0.04681,120.5080,85.5599,2.2373,1224.852,318.682383,464.460535,0.910106,-0.65240,0.26516,160.1819,-0.0226,1.50720,93.950155,95.615961,7.007087,1.891988,5.727343,5.919376,0.800760,0.9504,0.9456,0.0048,3.2704,0.6776,0.0023,3.5013,0.4318,-0.0354
4,2.log_files\pyrd5_conf-1,C3,N11,C10,C9,C4,C2,C1,H12,-441.213936,0.163471,-441.041089,0.042774,0.042581,-441.083863,-441.083670,298.15,-0.27694,-0.02704,-0.151990,0.24990,0.04622,122.7620,98.4500,2.0295,1101.884,324.352393,470.043193,0.901347,-0.63245,0.19098,158.9946,6.1303,1.50387,93.062371,95.725723,9.045144,1.922796,4.345787,5.969302,0.805719,0.9715,0.9514,0.0202,3.2801,0.6743,-0.0006,3.5869,0.3699,-0.0432
5,2.log_files\pyrd6_conf-1,C3,N4,C5,C6,C11,C2,C1,H12,-441.211694,0.163955,-441.038524,0.042194,0.042166,-441.080718,-441.080689,298.15,-0.27834,-0.02938,-0.153860,0.24896,0.04754,120.4170,86.1793,2.6858,1176.699,316.992285,463.193007,0.913293,-0.63265,-0.03253,167.9555,37.8733,1.50450,94.088972,96.736183,7.039571,1.888775,5.719279,6.099833,0.818823,1.0062,0.9630,0.0431,3.2651,0.6928,0.0044,3.8394,0.1259,-0.0076
6,2.log_files\pyrd7_conf-1,C10,C5,N4,C12,C11,C3,C2,H16,-480.522848,0.192010,-480.320158,0.046362,0.045507,-480.366519,-480.365665,298.15,-0.28450,-0.02491,-0.154705,0.25959,0.04610,135.4370,103.7670,1.7135,1314.468,354.252474,519.566637,0.882264,-0.43473,0.24166,151.3863,-2.2149,1.51424,97.465780,95.635331,8.904898,1.973969,4.536127,6.214157,0.829288,0.9466,0.9437,0.0029,3.4983,0.4552,-0.0107,3.5218,0.4184,-0.0388
7,2.log_files\pyrd7_conf-2,C10,C5,N4,C12,C11,C3,C2,H16,-480.522526,0.192231,-480.319616,0.046863,0.045637,-480.366479,-480.365253,298.15,-0.28386,-0.02761,-0.155735,0.25625,0.04732,136.0750,103.5480,1.7116,1259.791,354.512417,520.882133,0.883105,-0.43091,0.24023,147.7958,-6.3707,1.50957,97.084840,96.410124,8.865712,1.700000,4.635040,5.840125,0.792643,0.9492,0.9404,0.0088,3.4948,0.4592,

processing prefix: pyrmd
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
NMR function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
Pyramidalization function has completed for ['C1']
Natural Bond Order function has completed for [['C1', 'C2']]
Natural Atomic Valencies function has completed for ['C1', 'C2']


,log_name,N3,C4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NMR_shift_C1,NMR_shift_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrmd1_conf-1,N11,C10,C5,C4,N3,C2,C1,H12,-457.262567,0.151533,-457.101761,0.042710,0.042452,-457.144471,-457.144213,298.15,-0.29708,-0.03952,-0.168300,0.25756,0.05499,116.8870,93.5959,2.2131,1296.967,320.442033,460.149927,0.899499,-0.64668,0.41219,156.0842,-6.2961,1.50117,92.968750,94.482825,8.892560,1.924001,4.377494,5.812043,0.789449,0.9495,0.9418,0.0077,3.2653,0.6842,0.0059,3.2567,0.7021,-0.0412
1,2.log_files\pyrmd2_conf-1,C10,N9,C8,C3,N11,C2,C1,H12,-457.262070,0.151990,-457.100938,0.042228,0.042146,-457.143166,-457.143085,298.15,-0.29841,-0.04085,-0.169630,0.25756,0.05586,114.6880,82.1734,3.0521,1148.036,313.171933,453.652467,0.911696,-0.65969,0.30378,160.9985,-10.0707,1.50366,93.911415,95.683755,7.051690,1.888107,5.720280,5.882256,0.796943,0.9766,0.9594,0.0172,3.2620,0.7000,0.0189,3.4655,0.4795,-0.0465
2,2.log_files\pyrmd3_conf-1,N12,C11,C10,C5,N4,C3,C2,H16,-496.569414,0.180480,-496.378410,0.046118,0.045192,-496.424528,-496.423602,298.15,-0.29679,-0.03959,-0.168190,0.25720,0.05499,130.0170,99.2581,2.1909,1476.392,349.422054,509.596757,0.882981,-0.43804,0.41967,147.0026,-10.9898,1.50515,96.971849,94.999354,8.897868,1.870845,4.374501,5.919383,0.800924,0.9417,0.9315,0.0102,3.4869,0.4661,-0.0039,3.2303,0.7104,-0.0593
3,2.log_files\pyrmd3_conf-2,N4,C5,C10,C11,N12,C3,C2,H16,-496.569753,0.180221,-496.378993,0.045962,0.045234,-496.424955,-496.424227,298.15,-0.29699,-0.03857,-0.167780,0.25842,0.05447,129.2360,97.6600,2.2021,1521.085,349.072067,508.945758,0.883114,-0.44257,0.42082,149.3322,-9.0738,1.50798,97.317278,94.440857,8.912970,1.972342,4.349826,6.368443,0.843903,0.9361,0.9307,0.0054,3.4869,0.4666,-0.0056,3.2329,0.7073,-0.0598
4,2.log_files\pyrmd4_conf-1,C11,N10,C9,C4,N12,C3,C2,H16,-496.568315,0.180967,-496.376930,0.045464,0.044937,-496.422395,-496.421867,298.15,-0.29740,-0.04047,-0.168935,0.25693,0.05554,126.7680,80.8090,3.0319,1328.695,339.281889,500.546031,0.898572,-0.44720,0.31276,152.1763,-16.3750,1.50763,97.304365,96.374613,7.077407,1.718508,5.717085,5.887331,0.797150,0.9642,0.9524,0.0117,3.4878,0.4832,0.0066,3.4412,0.4820,-0.0532
5,2.log_files\pyrmd4_conf-2,C11,N10,C9,C4,N12,C3,C2,H16,-496.568846,0.180657,-496.377717,0.045601,0.045060,-496.423318,-496.422777,298.15,-0.29775,-0.03980,-0.168775,0.25795,0.05521,127.2010,88.4040,2.9273,1521.593,341.441984,501.785698,0.894361,-0.45049,0.31281,155.1883,-12.5165,1.51186,97.924199,95.683755,7.040337,1.974404,5.728233,6.193697,0.827347,0.9630,0.9572,0.0058,3.4945,0.4767,0.0049,3.4479,0.4768,-0.0528
6,2.log_files\pyrmd5_conf-1,N3,C4,C5,C6,N7,C2,C1,H8,-303.625303,0.104379,-303.513954,0.038046,0.037030,-303.552000,-303.550984,298.15,-0.32055,-0.01168,-0.166115,0.30887,0.04467,67.7395,39.8235,1.7125,979.820,252.681162,338.261455,0.929137,-0.64958,0.41284,156.9483,-11.8236,1.50268,92.904184,94.498967,6.774128,1.883393,3.269005,5.814303,0.789755,0.9510,0.9427,0.0083,3.2637,0.6873,0.0068,3.2422,0.6920,-0.0448
7,2.log_files\pyrmd6_conf-1,C4,N3,C7,C6,N5,C2,C1,H8,-303.625153,0.104624,-303.513626,0.037221,0.036653,-303.550847,-303.550278,298.15,-0.32427,-0.01133,-0.167800,0.31294,0.04499,67.6176,38.4641,2.5749,896.178,251.011067,337.015100,0.933020,-0.64526,0.24852,159.4124,-10.3481,1.50290,93.001033,95.745093,6.206459,1.885277,3.273264,5.920989,0.800838,0.9727,0.9585,0.0142,3.2783,0.

processing prefix: pyrz
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
NMR function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
Pyramidalization function has completed for ['C1']
Natural Bond Order function has completed for [['C1', 'C2']]
Natural Atomic Valencies function has completed for ['C1', 'C2']


,log_name,C3,N4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NMR_shift_C1,NMR_shift_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrz1_conf-1,C11,N10,C5,C4,N3,C2,C1,H12,-457.255348,0.151285,-457.094752,0.042943,0.042501,-457.137695,-457.137253,298.15,-0.29820,-0.04255,-0.170375,0.25565,0.05677,118.7930,100.9530,0.4099,1361.572,320.361982,460.245164,0.899848,-0.64689,0.18711,160.5351,6.6176,1.50366,93.110795,95.657929,8.894679,1.927542,4.362009,5.913906,0.800162,0.9605,0.9440,0.0165,3.2608,0.6942,0.0014,3.5551,0.3587,-0.0590
1,2.log_files\pyrz2_conf-1,C12,N11,C10,C5,N4,C3,C2,H16,-496.562505,0.180043,-496.371904,0.046266,0.045350,-496.418170,-496.417254,298.15,-0.29784,-0.04102,-0.169430,0.25682,0.05589,131.0300,104.2690,0.5136,1348.704,348.532017,508.465037,0.883925,-0.43756,0.19570,153.8126,3.6470,1.51101,97.446410,95.625646,8.916470,1.974528,4.332605,6.203055,0.828195,0.9416,0.9340,0.0076,3.4897,0.4708,-0.0064,3.5349,0.3586,-0.0704
2,2.log_files\pyrz2_conf-2,C12,N11,C10,C5,N4,C3,C2,H16,-496.562075,0.180255,-496.371262,0.046808,0.045503,-496.418069,-496.416764,298.15,-0.29758,-0.04463,-0.171105,0.25295,0.05787,131.7280,104.1640,0.5048,1361.670,348.761969,509.560324,0.884610,-0.43312,0.19348,151.3259,0.1784,1.50676,97.078383,96.429494,8.883435,1.700000,4.385299,5.849596,0.793623,0.9558,0.9401,0.0156,3.4853,0.4759,-0.0050,3.5339,0.3596,-0.0696
3,2.log_files\pyrz2_conf-3,C12,N11,C10,C5,N4,C3,C2,H16,-496.562075,0.180257,-496.371261,0.046799,0.045497,-496.418060,-496.416758,298.15,-0.29758,-0.04466,-0.171120,0.25292,0.05789,131.7250,104.1440,0.5042,1381.031,348.801966,509.616460,0.884574,-0.43308,0.19344,151.3308,0.2128,1.50678,97.055785,96.455320,8.883318,1.700000,4.386140,5.850815,0.793746,0.9557,0.9401,0.0156,3.4853,0.4759,-0.0050,3.5339,0.3596,-0.0696
4,2.log_files\pyrz3_conf-1,C7,N6,C5,C4,N3,C2,C1,H8,-303.616826,0.104395,-303.505551,0.037032,0.036574,-303.542583,-303.542126,298.15,-0.31665,-0.02123,-0.168940,0.29542,0.04831,68.6165,42.0286,0.5978,838.316,251.041073,337.090149,0.933047,-0.63965,0.16884,162.6439,4.6076,1.50329,93.059143,95.654700,6.774801,1.886680,3.252391,5.951607,0.804016,0.9645,0.9464,0.0181,3.2699,0.6883,0.0010,3.6053,0.3484,-0.0088
5,2.log_files\pyrz4_conf-1,C8,N7,C6,C5,N4,C3,C2,H12,-342.923914,0.133430,-342.782454,0.040057,0.039378,-342.822511,-342.821832,298.15,-0.31566,-0.02104,-0.168350,0.29462,0.04810,80.8982,43.9557,0.6487,995.062,280.451064,387.176885,0.916010,-0.42968,0.17543,152.8821,-0.8962,1.50674,97.046100,96.319731,6.777280,1.840731,3.264139,5.900098,0.798734,0.9472,0.9350,0.0122,3.4960,0.4711,-0.0115,3.5872,0.3417,-0.0218
6,2.log_files\pyrz4_conf-2,C8,N7,C6,C5,N4,C3,C2,H12,-342.922859,0.133400,-342.781484,0.039882,0.039210,-342.821366,-342.820694,298.15,-0.31452,-0.02027,-0.167395,0.29425,0.04761,80.6955,48.2870,0.7602,1042.427,277.620972,383.834322,0.920014,-0.43346,0.17538,153.6064,1.1660,1.51131,97.381844,95.725723,6.784918,1.974670,3.309893,6.327955,0.840274,0.9546,0.9389,0.0156,3.4956,0.4742,-0.0063,3.5927,0.3395,-0.0229


## 3.1 Save collected properties to Excel and pickle file

In [5]:
for prefix, df in atom_map_df_all.items():
    # save the pandas dataframe to a xlsx file
    with pd.ExcelWriter(temp_folder + os.sep + prefix + "_extracted_properties.xlsx") as writer:
        df.to_excel(writer)

# **4. Post-processing**

In [6]:
import re
import pandas as pd
import numpy as np
from tabulate import tabulate

In [7]:
# for numerically named compounds, prefix is any text common to all BEFORE the number and suffix is common to all AFTER the number
# this is a template for our files that are all named "AcXXX_clust-X.log" or "AcXXX_conf-X.log"
suffix = "_"
prefixs = {}
for file in glob.glob("*.xlsx", root_dir=atom_mappings_folder):
    key = re.search(r"^(\D+)_atom_map", file)
    if key and key.group(1) in prefixs:
        prefixs[key.group(1)].append(file)
    else:
        prefixs[key.group(1)] = [file]

# columns that provide atom mapping information are dropped, not need if these columns contain cells that cannot be convert to float
# e.g. atom_columns_to_drop = ["C3", "C4", "C5", "N1", "C1", "C2"]
atom_columns_to_drop = []

# title of the column for the energy you want to use for boltzmann averaging and lowest E conformer determination
energy_col_header = "G(T)_spc(Hartree)"

### Option to import an Excel sheet if you're using properties or energies collected outside of this notebook

##### If you would like to use post-processing functionality (i.e. Boltzmann averaging, lowest E conformers, etc.) you can read in a dataframe with properties (e.g. QikProp properties) or energies (e.g. if you don't/can't run linked jobs) collected outside of this notebook. 

In [8]:
atom_map_df_all = {}

for prefix in prefixs:
    df = pd.read_excel(
        temp_folder + os.sep +
        prefix + "_extracted_properties.xlsx",
        "Sheet1",
        index_col=0,
        header=0,
        engine="openpyxl",
    )
    display(df.head(2))
    atom_map_df_all[prefix] = df.copy(deep=True)

,log_name,N3,N4,C5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NMR_shift_C1,NMR_shift_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrdz1_conf-1,N7,N6,C5,C4,C3,C2,C1,H8,-303.585861,0.103728,-303.475215,0.036948,0.036617,-303.512163,-303.511832,298.15,-0.30541,-0.02096,-0.163185,0.28445,0.04681,68.2186,40.4381,4.1200,891.992,251.280909,337.591570,0.933080,-0.64306,0.17296,161.3275,-2.8831,1.50381,93.055914,95.674070,6.740781,1.843830,3.254861,5.896483,0.798456,0.9572,0.9355,0.0217,3.2503,0.6982,0.0083,3.6184,0.3130,-0.0686
1,2.log_files\pyrdz2_conf-1,C3,N4,N5,C6,C7,C2,C1,H8,-303.584349,0.103775,-303.473550,0.038012,0.037077,-303.511562,-303.510627,298.15,-0.30686,-0.01756,-0.162210,0.28930,0.04548,68.0596,38.9528,4.7174,870.504,250.420802,337.307042,0.935759,-0.63618,-0.01824,166.3383,27.3979,1.50329,93.188275,96.791064,6.241607,1.744658,3.257621,6.046751,0.813583,1.0035,0.9597,0.0438,3.2565,0.7099,0.0241,3.8146,0.1228,-0.0598


,log_name,C3,C4,N5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NMR_shift_C1,NMR_shift_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrd1_conf-1,C5,C4,N3,C11,C10,C2,C1,H12,-441.215801,0.163282,-441.043098,0.043006,0.042660,-441.086104,-441.085759,298.15,-0.28447,-0.02605,-0.155260,0.25842,0.04664,123.182,100.567,1.8012,1407.377,325.682431,470.864551,0.898712,-0.64364,0.23284,157.7232,0.4339,1.50642,93.055914,95.680527,8.877422,1.930717,4.598011,5.912687,0.800030,0.9552,0.9433,0.0120,3.2691,0.6784,0.0006,3.5377,0.4177,-0.0308
1,2.log_files\pyrd2_conf-1,C11,N10,C9,C4,C3,C2,C1,H12,-441.212895,0.163425,-441.040035,0.043108,0.042693,-441.083144,-441.082728,298.15,-0.28330,-0.02685,-0.155075,0.25645,0.04689,123.175,102.427,2.2201,1371.628,324.632344,470.260010,0.900847,-0.62780,-0.06631,164.9883,32.8402,1.50530,93.356147,96.742639,9.049449,1.924579,4.320338,6.092525,0.818093,1.0016,0.9637,0.0378,3.2747,0.6890,-0.0078,3.7837,0.1419,-0.0467


,log_name,N3,C4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NMR_shift_C1,NMR_shift_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrmd1_conf-1,N11,C10,C5,C4,N3,C2,C1,H12,-457.262567,0.151533,-457.101761,0.042710,0.042452,-457.144471,-457.144213,298.15,-0.29708,-0.03952,-0.16830,0.25756,0.05499,116.887,93.5959,2.2131,1296.967,320.442033,460.149927,0.899499,-0.64668,0.41219,156.0842,-6.2961,1.50117,92.968750,94.482825,8.89256,1.924001,4.377494,5.812043,0.789449,0.9495,0.9418,0.0077,3.2653,0.6842,0.0059,3.2567,0.7021,-0.0412
1,2.log_files\pyrmd2_conf-1,C10,N9,C8,C3,N11,C2,C1,H12,-457.262070,0.151990,-457.100938,0.042228,0.042146,-457.143166,-457.143085,298.15,-0.29841,-0.04085,-0.16963,0.25756,0.05586,114.688,82.1734,3.0521,1148.036,313.171933,453.652467,0.911696,-0.65969,0.30378,160.9985,-10.0707,1.50366,93.911415,95.683755,7.05169,1.888107,5.720280,5.882256,0.796943,0.9766,0.9594,0.0172,3.2620,0.7000,0.0189,3.4655,0.4795,-0.0465


,log_name,C3,N4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,NMR_shift_C1,NMR_shift_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,pyramidalization_Gavrish_C1(°),pyramidalization_Agranat-Radhakrishnan_C1,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrz1_conf-1,C11,N10,C5,C4,N3,C2,C1,H12,-457.255348,0.151285,-457.094752,0.042943,0.042501,-457.137695,-457.137253,298.15,-0.29820,-0.04255,-0.170375,0.25565,0.05677,118.793,100.953,0.4099,1361.572,320.361982,460.245164,0.899848,-0.64689,0.18711,160.5351,6.6176,1.50366,93.110795,95.657929,8.894679,1.927542,4.362009,5.913906,0.800162,0.9605,0.944,0.0165,3.2608,0.6942,0.0014,3.5551,0.3587,-0.0590
1,2.log_files\pyrz2_conf-1,C12,N11,C10,C5,N4,C3,C2,H16,-496.562505,0.180043,-496.371904,0.046266,0.045350,-496.418170,-496.417254,298.15,-0.29784,-0.04102,-0.169430,0.25682,0.05589,131.030,104.269,0.5136,1348.704,348.532017,508.465037,0.883925,-0.43756,0.19570,153.8126,3.6470,1.51101,97.446410,95.625646,8.916470,1.974528,4.332605,6.203055,0.828195,0.9416,0.934,0.0076,3.4897,0.4708,-0.0064,3.5349,0.3586,-0.0704


## 4.1 Generating a list of compounds that have conformational ensembles

**ONLY RUN THE AUTOMATED OR THE MANUAL CELL, NOT BOTH**

**AUTOMATED:** if your compounds are named consistenly, this section generates your compound list based on the similar naming structure

In [9]:
compound_list_all = {}

for prefix, df in atom_map_df_all.items():
    print(f"processing prefix: {prefix}")
    compound_list = []

    for index, row in df.iterrows():
        log_file = row["log_name"]  # read file name from df
        # first split by "\" take the last part
        log_file = log_file.split(os.sep)[-1]
        prefix_and_compound = log_file.split(str(suffix))
        compound = prefix_and_compound[0].split(str(prefix))  # splits again to get "XXX" (entry 1) (and we don't use the empty string "" (entry 0))
        compound_list.append(compound[1])

    compound_list = list(set(compound_list))  # removes duplicate stuctures that result from having conformers of each
    compound_list.sort(key=lambda x: int(re.search(r"\d+", x).group()))  # reorders numerically (not sure if it reorders alphabetically)
    print(f"items numbering: {compound_list}")
    compound_list_all[prefix] = compound_list

    # this should generate a list that looks like this: ['24', '27', '34', '48']

processing prefix: pyrdz
items numbering: ['1', '2', '3']
processing prefix: pyrd
items numbering: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']
processing prefix: pyrmd
items numbering: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
processing prefix: pyrz
items numbering: ['1', '2', '3', '4']


## 4.2 Post-processing to get properties for each compound

##### changes made in 8/30/2024 <br> 1. avoid divide by zero error in the Boltzmann averaging, the original code had the if block order reversed, which caused the error. <br> 2. data cleaning by remove columns contain cell that cannot be converted to float. <br> 3. concat all data into row before concat them into the final dataframe. The originl modify individual cells which result in fragmented and raise performance warning.

In [10]:
all_df_master_for_all_prefixes = {}
properties_df_master_for_all_prefixes = {}

for prefix, df in atom_map_df_all.items():
    compound_list = compound_list_all[prefix]
    all_df_master = pd.DataFrame(columns=[])
    properties_df_master = pd.DataFrame(columns=[])

    for compound in compound_list:
        # defines the common start to all files using the input above
        substring = log_files_folder + os.sep + str(prefix) + str(compound) + str(suffix)

        # makes a data frame for one compound at a time for post-processing
        valuesdf = df[df["log_name"].str.startswith(substring)]
        valuesdf = valuesdf.drop(columns=atom_columns_to_drop)
        valuesdf = valuesdf.reset_index(
            drop=True
        )  # you must re-index otherwise the 2nd, 3rd, etc. compounds fail

        # filter column that are characters, we will attempt to convert them to numeric numbers, if fail, we will drop them
        for column in valuesdf:
            try:
                # exclude column "log_name"
                if column == "log_name":
                    continue
                valuesdf[column] = pd.to_numeric(valuesdf[column])
            except:
                print(f"Column {column} contains non-numeric values")
                valuesdf = valuesdf.drop(columns=column)
                valuesdf = valuesdf.reset_index(
                    drop=True
                )  # reset the index after dropping columns

        # define columns that won't be included in summary properties or are treated differently because they don't make sense to Boltzmann average
        non_boltz_columns = [
            "G(Hartree)",
            "∆G(Hartree)",
            "∆G(kcal/mol)",
            "e^(-∆G/RT)",
            "Mole Fraction",
        ]  # don't boltzman average columns containing these strings in the column label
        reg_avg_columns = [
            "CPU_time_total(hours)",
            "Wall_time_total(hours)",
        ]  # don't boltzmann average these either, we average them in case that is helpful
        gv_extra_columns = [
            "G(T)_spc(Hartree)",
        ]
        gv_extra_columns.remove(str(energy_col_header))

        # calculate the summary properties based on all conformers (Boltzmann Average, Minimum, Maximum, Boltzmann Weighted Std)
        valuesdf["∆G(Hartree)"] = (
            valuesdf[energy_col_header] - valuesdf[energy_col_header].min()
        )
        valuesdf["∆G(kcal/mol)"] = valuesdf["∆G(Hartree)"] * 627.5
        valuesdf["e^(-∆G/RT)"] = np.exp(
            (valuesdf["∆G(kcal/mol)"] * -1000) / (1.987204 * 298.15)
        )  # R is in cal/(K*mol)
        valuesdf["Mole Fraction"] = valuesdf["e^(-∆G/RT)"] / valuesdf["e^(-∆G/RT)"].sum()
        values_boltz_row = []
        values_min_row = []
        values_max_row = []
        values_boltz_stdev_row = []
        values_range_row = []
        values_exclude_columns = []

        for column in valuesdf:
            if "log_name" in column:
                values_boltz_row.append("Boltzmann Averages")
                values_min_row.append("Ensemble Minimum")
                values_max_row.append("Ensemble Maximum")
                values_boltz_stdev_row.append("Boltzmann Standard Deviation")
                values_range_row.append("Ensemble Range")
                values_exclude_columns.append(column)  # used later to build final dataframe
            elif any(phrase in column for phrase in non_boltz_columns) or any(
                phrase in column for phrase in gv_extra_columns
            ):
                values_boltz_row.append("")
                values_min_row.append("")
                values_max_row.append("")
                values_boltz_stdev_row.append("")
                values_range_row.append("")
            elif any(phrase in column for phrase in reg_avg_columns):
                values_boltz_row.append(
                    valuesdf[column].mean()
                )  # intended to print the average CPU/wall time in the boltz column
                values_min_row.append("")
                values_max_row.append("")
                values_boltz_stdev_row.append("")
                values_range_row.append("")
            else:
                valuesdf[column] = pd.to_numeric(
                    valuesdf[column]
                )  # to hopefully solve the error that sometimes occurs where the float(Mole Fraction) cannot be mulitplied by the string(property)
                values_boltz_row.append(
                    (valuesdf[column] * valuesdf["Mole Fraction"]).sum()
                )
                values_min_row.append(valuesdf[column].min())
                values_max_row.append(valuesdf[column].max())
                values_range_row.append(valuesdf[column].max() - valuesdf[column].min())

                # this section generates the weighted std deviation (weighted by mole fraction)
                # formula: https://www.statology.org/weighted-standard-deviation-excel/

                boltz = (valuesdf[column] * valuesdf["Mole Fraction"]).sum()  # number
                delta_values_sq = []

                # makes a list of the "deviation" for each conformer
                for index, row in valuesdf.iterrows():
                    value = row[column]
                    delta_value_sq = (value - boltz) ** 2
                    delta_values_sq.append(delta_value_sq)

                # w is list of weights (i.e. mole fractions)
                w = list(valuesdf["Mole Fraction"])
                # !swap the order here to avoid division by zero error
                if (
                    len(w) == 1
                ):  # if there is only one conformer in the ensemble, set the weighted standard deviation to 0
                    wstdev = 0
                # np.average(delta_values_sq, weights=w) generates sum of each (delta_value_sq * mole fraction)
                else:
                    wstdev = np.sqrt(
                        (np.average(delta_values_sq, weights=w))
                        / (((len(w) - 1) / len(w)) * np.sum(w))
                    )
                values_boltz_stdev_row.append(wstdev)

        valuesdf.loc[len(valuesdf)] = values_boltz_row
        valuesdf.loc[len(valuesdf)] = values_boltz_stdev_row
        valuesdf.loc[len(valuesdf)] = values_min_row
        valuesdf.loc[len(valuesdf)] = values_max_row
        valuesdf.loc[len(valuesdf)] = values_range_row

        # final output format is built here:
        explicit_order_front_columns = [
            "log_name",
            energy_col_header,
            "∆G(Hartree)",
            "∆G(kcal/mol)",
            "e^(-∆G/RT)",
            "Mole Fraction",
        ]

        # reorders the dataframe using front columns defined above
        valuesdf = valuesdf[
            explicit_order_front_columns
            + [
                col
                for col in valuesdf.columns
                if col not in explicit_order_front_columns
                and col not in values_exclude_columns
            ]
        ]

        # determine the index of the lowest energy conformer
        low_e_index = valuesdf[valuesdf["∆G(Hartree)"] == 0].index.tolist()
        # copy the row to a new_row with the name of the log changed to Lowest E Conformer
        new_row = pd.DataFrame(valuesdf.loc[low_e_index[0]]).T
        new_row["log_name"] = "Lowest E Conformer"

        valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)

        # ------------------------------EDIT THIS SECTION IF YOU WANT A SPECIFIC CONFORMER----------------------------------
        # if you want all properties for a conformer with a particular property (i.e. all properties for the Vbur_min conformer)
        # this template can be adjusted for min/max/etc.

        # find the index for the min or max column:
        ensemble_min_index = valuesdf[
            valuesdf["log_name"] == "Ensemble Minimum"
        ].index.tolist()

        # find the min or max value of the property (based on index above)
        # saves the value in a list (min_value) with one entry (this is why we call min_value[0])
        min_value = valuesdf.loc[ensemble_min_index, "%Vbur_C1_2.0Å"].tolist()
        vbur_min_index = valuesdf[valuesdf["%Vbur_C1_2.0Å"] == min_value[0]].index.tolist()

        # copy the row to a new_row with the name of the log changed to Property_min_conformer
        new_row = pd.DataFrame(valuesdf.loc[vbur_min_index[0]]).T
        new_row["log_name"] = "%Vbur_C1_2.0Å_min_Conformer"

        valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)

        # --------------------------------------------------------------------------------------------------------------------

        # !here we define a list of properties we only want the minimal value for
        min_property_list = [
            "E_spc (Hartree)",
            "H_spc(Hartree)",
            "T",
            "T*S",
            "T*qh_S",
            "ZPE(Hartree)",
            "qh_G(T)_spc(Hartree)",
            "G(T)_spc(Hartree)",
        ]
        # extract the "Lowest E Conformer" row out of the dataframe
        Low_E_Conformer_row = pd.DataFrame(
            valuesdf.loc[valuesdf["log_name"] == "Lowest E Conformer"]
        )
        # extract the "Boltzmann Averages" row out of the dataframe
        Boltz_Avg_row = pd.DataFrame(
            valuesdf.loc[valuesdf["log_name"] == "Boltzmann Averages"]
        )
        # display(valuesdf) # debug display for finding the row index
        # display(Low_E_Conformer_row)

        # appends the frame to the master output
        all_df_master = pd.concat([all_df_master, valuesdf])

        # drop all the individual conformers
        dropindex = valuesdf[valuesdf["log_name"].str.startswith(substring)].index
        valuesdf = valuesdf.drop(dropindex)
        valuesdf = valuesdf.reset_index(drop=True)

        # drop the columns created to determine the mole fraction and some that
        valuesdf = valuesdf.drop(columns=explicit_order_front_columns)
        try:
            valuesdf = valuesdf.drop(columns=gv_extra_columns)
        except:
            pass
        try:
            valuesdf = valuesdf.drop(columns=reg_avg_columns)
        except:
            pass

        # ---------------------THIS MAY NEED TO CHANGE DEPENDING ON HOW YOU LABEL YOUR COMPOUNDS------------------------------
        compound_name = prefix + str(compound)
        # --------------------------------------------------------------------------------------------------------------------

        properties_df = pd.DataFrame({"Compound_Name": [compound_name]})

        # builds a dataframe (for each compound) by adding summary properties as new columns
        for column in valuesdf:
            # print(column)
            # the indexes need to match the values dataframe - display it to double check if you need to make changes
            # (uncomment the display(valuesdf) in row 124 of this cell)

            # create a list of headers for the properties_df
            # if you're collecting properties for a specific conformer, edit the header to reflect that, it should match the order in the valuesdf log_name column
            if column in min_property_list:
                # ! if we are working with a property that we only want the minimum value for, we only need one header
                headers = [
                    f"{column}",
                ]
                # use data from the Low_E_Conformer_row
                row_dataframe = pd.DataFrame(
                    [Low_E_Conformer_row[column].values], columns=headers
                )
            else:
                headers = [
                    f"{column}_Boltz",
                ]
                row_dataframe = pd.DataFrame([Boltz_Avg_row[column].values], columns=headers)
            # Extract values for the current column from valuesdf and create a DataFrame
            # Display the DataFrame for verification
            # display(row_dataframe)
            # Concatenate the new DataFrame to the properties_df along the columns (axis=1)
            properties_df = pd.concat([properties_df, row_dataframe], axis=1)

        # concatenates the individual acid properties df into the master properties df
        properties_df_master = pd.concat([properties_df_master, properties_df], axis=0)

    # Reset the index of the master DataFrames
    all_df_master = all_df_master.reset_index(drop=True)
    all_df_master_for_all_prefixes[prefix] = all_df_master.copy(deep=True)
    properties_df_master = properties_df_master.reset_index(drop=True)
    properties_df_master_for_all_prefixes[prefix] = properties_df_master.copy(deep=True)
    
    # Print in tabulated format
    print(tabulate(properties_df_master_for_all_prefixes[prefix], headers="keys", tablefmt="pretty"))
    print(tabulate(all_df_master_for_all_prefixes[prefix], headers="keys", tablefmt="pretty"))

Column N3 contains non-numeric values
Column N4 contains non-numeric values
Column C5 contains non-numeric values
Column C6 contains non-numeric values
Column C7 contains non-numeric values
Column C2 contains non-numeric values
Column C1 contains non-numeric values
Column H1 contains non-numeric values
Column N3 contains non-numeric values
Column N4 contains non-numeric values
Column C5 contains non-numeric values
Column C6 contains non-numeric values
Column C7 contains non-numeric values
Column C2 contains non-numeric values
Column C1 contains non-numeric values
Column H1 contains non-numeric values
Column N3 contains non-numeric values
Column N4 contains non-numeric values
Column C5 contains non-numeric values
Column C6 contains non-numeric values
Column C7 contains non-numeric values
Column C2 contains non-numeric values
Column C1 contains non-numeric values
Column H1 contains non-numeric values
+---+---------------+-----------------+--------------+----------------+----------+------

In [11]:
# merge all the properties_df_master_for_all_prefixes into a single dataframe, combine column with the same name
properties_df_master_for_all_prefixes_merged = pd.DataFrame(columns=[])
for prefix, df in properties_df_master_for_all_prefixes.items():
    display(df.head(1))
    properties_df_master_for_all_prefixes_merged = pd.concat([properties_df_master_for_all_prefixes_merged, df], axis=0)
    properties_df_master_for_all_prefixes_merged.reset_index(drop=True, inplace=True)

print(f"Combine summary properties for all prefixes: ")
display(properties_df_master_for_all_prefixes_merged.head(5))

,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,NMR_shift_C1_Boltz,NMR_shift_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz,pyramidalization_Gavrish_C1(°)_Boltz,pyramidalization_Agranat-Radhakrishnan_C1_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrdz1,-303.585861,0.103728,-303.475215,0.036948,0.036617,-303.511832,298.15,-0.30541,-0.02096,-0.163185,0.28445,0.04681,68.2186,40.4381,4.12,891.992,251.280909,337.59157,0.93308,-0.64306,0.17296,161.3275,-2.8831,1.50381,93.055914,95.67407,6.740781,1.84383,3.254861,5.896483,0.798456,0.9572,0.9355,0.0217,3.2503,0.6982,0.0083,3.6184,0.313,-0.0686


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,NMR_shift_C1_Boltz,NMR_shift_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz,pyramidalization_Gavrish_C1(°)_Boltz,pyramidalization_Agranat-Radhakrishnan_C1_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrd1,-441.215801,0.163282,-441.043098,0.043006,0.04266,-441.085759,298.15,-0.28447,-0.02605,-0.15526,0.25842,0.04664,123.182,100.567,1.8012,1407.377,325.682431,470.864551,0.898712,-0.64364,0.23284,157.7232,0.4339,1.50642,93.055914,95.680527,8.877422,1.930717,4.598011,5.912687,0.80003,0.9552,0.9433,0.012,3.2691,0.6784,0.0006,3.5377,0.4177,-0.0308


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,NMR_shift_C1_Boltz,NMR_shift_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz,pyramidalization_Gavrish_C1(°)_Boltz,pyramidalization_Agranat-Radhakrishnan_C1_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrmd1,-457.262567,0.151533,-457.101761,0.04271,0.042452,-457.144213,298.15,-0.29708,-0.03952,-0.1683,0.25756,0.05499,116.887,93.5959,2.2131,1296.967,320.442033,460.149927,0.899499,-0.64668,0.41219,156.0842,-6.2961,1.50117,92.96875,94.482825,8.89256,1.924001,4.377494,5.812043,0.789449,0.9495,0.9418,0.0077,3.2653,0.6842,0.0059,3.2567,0.7021,-0.0412


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,NMR_shift_C1_Boltz,NMR_shift_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz,pyramidalization_Gavrish_C1(°)_Boltz,pyramidalization_Agranat-Radhakrishnan_C1_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrz1,-457.255348,0.151285,-457.094752,0.042943,0.042501,-457.137253,298.15,-0.2982,-0.04255,-0.170375,0.25565,0.05677,118.793,100.953,0.4099,1361.572,320.361982,460.245164,0.899848,-0.64689,0.18711,160.5351,6.6176,1.50366,93.110795,95.657929,8.894679,1.927542,4.362009,5.913906,0.800162,0.9605,0.944,0.0165,3.2608,0.6942,0.0014,3.5551,0.3587,-0.059


Combine summary properties for all prefixes: 


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,NMR_shift_C1_Boltz,NMR_shift_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz,pyramidalization_Gavrish_C1(°)_Boltz,pyramidalization_Agranat-Radhakrishnan_C1_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrdz1,-303.585861,0.103728,-303.475215,0.036948,0.036617,-303.511832,298.15,-0.305410,-0.020960,-0.163185,0.28445,0.046810,68.218600,40.438100,4.120000,891.992000,251.280909,337.591570,0.933080,-0.643060,0.172960,161.327500,-2.883100,1.50381,93.055914,95.674070,6.740781,1.843830,3.254861,5.896483,0.798456,0.957200,0.935500,0.021700,3.250300,0.698200,0.00830,3.618400,0.313000,-0.068600
1,pyrdz2,-303.584349,0.103775,-303.473550,0.038012,0.037077,-303.510627,298.15,-0.306860,-0.017560,-0.162210,0.28930,0.045480,68.059600,38.952800,4.717400,870.504000,250.420802,337.307042,0.935759,-0.636180,-0.018240,166.338300,27.397900,1.50329,93.188275,96.791064,6.241607,1.744658,3.257621,6.046751,0.813583,1.003500,0.959700,0.043800,3.256500,0.709900,0.02410,3.814600,0.122800,-0.059800
2,pyrdz3,-342.892888,0.132713,-342.752044,0.040598,0.039684,-342.791728,298.15,-0.303892,-0.020262,-0.162077,0.28363,0.046308,80.375365,44.261098,3.988204,1030.461083,279.277152,386.172806,0.918271,-0.431648,0.179649,152.407698,-8.327977,1.51035,97.198191,95.950567,6.743305,1.858882,3.261320,6.096538,0.817779,0.948079,0.932185,0.015852,3.478598,0.475177,-0.00134,3.601762,0.314021,-0.069421
3,pyrd1,-441.215801,0.163282,-441.043098,0.043006,0.042660,-441.085759,298.15,-0.284470,-0.026050,-0.155260,0.25842,0.046640,123.182000,100.567000,1.801200,1407.377000,325.682431,470.864551,0.898712,-0.643640,0.232840,157.723200,0.433900,1.50642,93.055914,95.680527,8.877422,1.930717,4.598011,5.912687,0.800030,0.955200,0.943300,0.012000,3.269100,0.678400,0.00060,3.537700,0.417700,-0.030800
4,pyrd2,-441.212895,0.163425,-441.040035,0.043108,0.042693,-441.082728,298.15,-0.283300,-0.026850,-0.155075,0.25645,0.046890,123.175000,102.427000,2.220100,1371.628000,324.632344,470.260010,0.900847,-0.627800,-0.066310,164.988300,32.840200,1.50530,93.356147,96.742639,9.049449,1.924579,4.320338,6.092525,0.818093,1.001600,0.963700,0.037800,3.274700,0.689000,-0.00780,3.783700,0.141900,-0.046700


# 5. Export the data

In [12]:
# Define the filename for the Excel file
with pd.ExcelWriter(output_folder + os.sep + "Properties_postprocessed_all_prefixes.xlsx", engine="xlsxwriter") as writer:
    for prefix, properties_df_master in properties_df_master_for_all_prefixes.items():
        print(f"Writing to Excel file for prefix: {prefix}")
        all_df_master = all_df_master_for_all_prefixes[prefix]

        all_df_master.to_excel(writer, sheet_name="All_Conformer_Properties_" + prefix, index=False)
        # automatically adjusts the width of the columns
        for column in all_df_master.columns:
            column_width = max(all_df_master[column].astype(str).map(len).max(), len(column))
            col_idx = all_df_master.columns.get_loc(column)
            writer.sheets["All_Conformer_Properties_" + prefix].set_column(col_idx, col_idx, column_width)
        properties_df_master.to_excel(writer, sheet_name="Summary_Properties_" + prefix, index=False)
        # automatically adjusts the width of the columns
        for column in properties_df_master.columns:
            column_width = max(properties_df_master[column].astype(str).map(len).max(), len(column))
            col_idx = properties_df_master.columns.get_loc(column)
            writer.sheets["Summary_Properties_" + prefix].set_column(col_idx, col_idx, column_width)

Writing to Excel file for prefix: pyrdz
Writing to Excel file for prefix: pyrd
Writing to Excel file for prefix: pyrmd
Writing to Excel file for prefix: pyrz


In [13]:
# write the combined properties_df_master_for_all_prefixes to an Excel file
with pd.ExcelWriter(output_folder + os.sep + "Summary_Properties_all.xlsx", engine="xlsxwriter") as writer:
    properties_df_master_for_all_prefixes_merged.to_excel(writer, sheet_name="Summary_Properties", index=False)
    # automatically adjusts the width of the columns
    for column in properties_df_master_for_all_prefixes_merged.columns:
        column_width = max(properties_df_master_for_all_prefixes_merged[column].astype(str).map(len).max(), len(column))
        col_idx = properties_df_master_for_all_prefixes_merged.columns.get_loc(column)
        writer.sheets["Summary_Properties"].set_column(col_idx, col_idx, column_width)